# Data Joins
Joing the multiple CSVs from the GPRA coding project due to the amount of pyton scripts fragmenting the data frame

In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime

In [2]:
#Importing data
GPRA1 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA1_STAR.csv", parse_dates = ['InterviewDate'])
GPRA2 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA2_STAR.csv")
GPRA3 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA3_STAR.csv")
GPRA4 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA4_STAR.csv")
GPRA5 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA5_STAR.csv")
GPRA6 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA6_STAR.csv")
client_id = pd.read_excel(r"W:\Research and Evaluation\STAR\GPRA\GPRA Client ID Master List_Full.xlsx",sheet_name = 'List')


In [3]:
#Merging all of the GPRA files
Full_GPRA_File = GPRA1.merge(GPRA2, on = 'ClientID', how = 'left')\
            .merge(GPRA3, on = 'ClientID', how = 'left')\
            .merge(GPRA4, on = 'ClientID', how = 'left')\
            .merge(GPRA5, on = 'ClientID', how = 'left')\
            .merge(GPRA6, on = 'ClientID', how = 'left')

In [4]:
# dropping duplicates
Full_GPRA_File = Full_GPRA_File.drop_duplicates()

In [5]:
#Adding ClientOD
client_id = client_id[['PATIENT','GPRA ID']] # <-Subsetting
client_id.rename(columns = {'GPRA ID':'ClientID'}, inplace = True)

In [6]:
#Subsetting dataset for merge to create IntakeDate Column 
Intake_Date_Merge = Full_GPRA_File[['ClientID','InterviewType','InterviewDate']]

#Filtering dataset for intake
Intake_Date_Merge = Intake_Date_Merge[Intake_Date_Merge['InterviewType'] == 1]

#Subsetting dataset further
Intake_Date_Merge = Intake_Date_Merge[['ClientID', 'InterviewDate']]

#Renaming columns
Intake_Date_Merge.rename(columns = {'InterviewDate':'IntakeDate'}, inplace = True)

#Merging files
Full_GPRA_File_STAR = Full_GPRA_File.merge(Intake_Date_Merge, on= 'ClientID', how = 'left')

# Merging file to return client_id
Full_GPRA_File_STAR = Full_GPRA_File_STAR.merge(client_id, on = 'PATIENT', how = 'left') # <- returning ClientIDs

Full_GPRA_File_STAR = Full_GPRA_File_STAR[Full_GPRA_File_STAR['PROVIDER_COUNTY'] != 'Monmouth'] # <- excluding Monmouth County

In [7]:
Full_GPRA_File_STAR['PROVIDER_COUNTY'].unique() # <- QA Provider County

array(['Hudson', 'Essex', 'Middlesex', nan], dtype=object)

## Current Upload: 09.09.24- 10.06.24

## Next Upload: 

In [8]:
# Date Filter for current date range and eliminating non interviews
Full_GPRA_File_STAR = Full_GPRA_File_STAR[(Full_GPRA_File_STAR['InterviewDate']>= '2024-09-09') & (Full_GPRA_File_STAR['ConductedInterview'] != 0)]


In [9]:
# Full_GPRA_File_STAR = Full_GPRA_File_STAR[Full_GPRA_File_STAR['ConductedInterview'] != 0]

In [10]:
#Exporting GPRA full file

Full_GPRA_File_STAR.to_excel('Full_GPRA_File_STAR.xlsx', index = False, header = True)
